## Module 2 - Deep MNIST


This is step-by-step follow along based on Google's Tensorflow Tutorial.

[Deep MNIST for Experts](https://www.tensorflow.org/get_started/mnist/pros)


In this tutorial we will learn the basic building blocks of a TensorFlow model while constructing a deep convolutional MNIST classifier. What we will do:



    Create a softmax regression function that is a model for recognizing MNIST digits, based on looking at every pixel in the image

    Use Tensorflow to train the model to recognize digits by having it "look" at thousands of examples (and run our first Tensorflow session to do so)

    Check the model's accuracy with our test data

    Build, train, and test a multilayer convolutional neural network to improve the results


## Load MNIST Data

Here mnist is a lightweight class which stores the training, validation, and testing sets as NumPy arrays. It also provides a function for iterating through data minibatches.

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Start TensorFlow InteractiveSession

TensorFlow relies on a highly efficient C++ backend to do its computation. The connection to this backend is called a session. The common usage for TensorFlow programs is to first create a graph and then launch it in a session.

InteractiveSession class allows you to interleave operations which build a computation graph with ones that run the graph. This is particularly convenient when working in interactive contexts like IPython. 

If you are not using an InteractiveSession, then you should build the entire computation graph before starting a session and launching the graph.

In [2]:
import tensorflow as tf
sess = tf.InteractiveSession()


## Computation Graph

To do efficient numerical computing in Python, we typically use libraries like NumPy that do expensive operations such as matrix multiplication outside Python, using highly efficient code implemented in another language. Unfortunately, there is a lot of overhead from switching back to Python every operation. 

TensorFlow also does its heavy lifting outside Python. To avoid this overhead, instead of running a single expensive operation independently from Python, TensorFlow lets us describe a graph of interacting operations that run entirely outside Python. This approach is similar to that used in Theano or Torch.

The role of the Python code is therefore to build this external computation graph, and to dictate which parts of the computation graph should be run.

## Set up the Model

We will build a softmax regression model with a single linear layer.

We build the computation graph by creating nodes for the input images and target output classes (x,y).They are each a placeholder -- a value that we'll input when we ask TensorFlow to run a computation.

Next we define define the weights W and biases b for our model. In machine learning applications, we always have the model parameters be Variables.

Next we implement the regression model and the loss function.

In [4]:
# The input images x will consist of a 2d tensor of floating point numbers with a shape of [None, 784], 
# where 784 is the dimensionality of a single flattened 28 by 28 pixel MNIST image, and None indicates
# that the first dimension, corresponding to the batch size, can be of any size.
x = tf.placeholder(tf.float32, shape=[None, 784])
# The target output classes y_ will also consist of a 2d tensor, where each row is a one-hot 10-d vector
y_ = tf.placeholder(tf.float32, shape=[None, 10])

# The model parameters W and b are set up as Variables
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
sess.run(tf.global_variables_initializer())  # Remember to initialize them

# Regression model and the loss function.
y = tf.matmul(x,W) + b
# tf.nn.softmax_cross_entropy_with_logits internally applies the softmax on the model's unnormalized model 
# prediction and sums across all classes, and tf.reduce_mean takes the average over these sums.
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))


## Train the Model

After defining our model and training loss function, TensorFlow knows the entire computation graph so it can use automatic differentiation to find the gradients of the loss with respect to each of the variables. It does this with a variety of built-in optimization algorithms. In this case, we use steepest gradient descent, with a step length of 0.5, to descend the cross entropy.

The returned operation train_step, when run, will apply the gradient descent updates to the parameters. Training the model can therefore be accomplished by repeatedly running train_step.

In [5]:
# What TensorFlow actually did in this single line was to add new operations to the computation graph. These
# operations included ones to compute gradients, compute parameter update steps, and apply update steps to 
# the parameters.
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# We run train_step 1000 times
for _ in range(1000):
  # We load 100 training examples in each training iteration. We then run the train_step operation, using 
  # feed_dict to replace the placeholder tensors x and y_ with the training examples.  
  batch = mnist.train.next_batch(100)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})


## Evaluate the Model

In [6]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))


0.9196


## Multilayer Convolutional Network

Getting 92% accuracy on MNIST is bad. In this section, we'll fix that, jumping from a very simple model to something moderately sophisticated: a small convolutional neural network. This will get us to around 99.2% accuracy -- not state of the art, but respectable.

## Weight Initialization

Since we're using ReLU neurons, it is also good practice to initialize them with a slightly positive initial bias to avoid "dead neurons". Instead of doing this repeatedly while we build the model, we will create two handy functions to do it for us.

In [7]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)


## Convolution and Pooling

TensorFlow gives us a lot of flexibility in convolution and pooling operations. 

Our convolutions uses a stride of one and are zero padded so that the output is the same size as the input. Our pooling is plain old max pooling over 2x2 blocks. To keep our code cleaner, let's also abstract those operations into functions.

In [8]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

## First Convolutional Layer

In [9]:
# We first reshape x to a 4d tensor
x_image = tf.reshape(x, [-1,28,28,1])

# The convolution will compute 32 features for each 5x5 patch. Its weight tensor will have a shape 
# of [5, 5, 1, 32]. We will also have a bias vector with a component for each output channel.
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# We then convolve x_image with the weight tensor, add the bias, apply the ReLU function, then max pool. 
# The max_pool_2x2 method will reduce the image size to 14x14.
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)


## Second Convolutional Layer

In [10]:
# The second layer will have 64 features for each 5x5 patch.
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

# The image size will be reduced by maxpooling to 7x7
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

## Fully Connected Layer

In [11]:
# We add a fully-connected layer with 1024 neurons to allow processing on the entire image. 
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

# We reshape the tensor from the pooling layer into a batch of vectors, multiply by a weight matrix, 
# add a bias, and apply a ReLU.
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


## Dropout

To reduce overfitting, we will apply dropout before the readout layer. 

In [12]:

# We create a placeholder for the probability that a neuron's output is kept during dropout. 
# This allows us to turn dropout on during training, and turn it off during testing.
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


## Readout

In [13]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

## Train and Evaluate Model

The code is almost identical to the simpler model above. The differences are that:

    We will replace the steepest gradient descent optimizer with the more sophisticated ADAM optimizer.

    We will include the additional parameter keep_prob in feed_dict to control the dropout rate.

    We will add logging to every 100th iteration in the training process.


In [14]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.global_variables_initializer())

for i in range(20000):
  batch = mnist.train.next_batch(50)
  # Log every 100th iteration 
  if i%100 == 0:
    # Note that dropout is zero when we evaluate training accuracy
    train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))

  # Dropout is 50% during training
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

# Note that dropout is zero when we evaluate testing accuracy    
print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))


step 0, training accuracy 0.1
step 100, training accuracy 0.82
step 200, training accuracy 0.88
step 300, training accuracy 0.8
step 400, training accuracy 0.94
step 500, training accuracy 0.96
step 600, training accuracy 0.94
step 700, training accuracy 0.96
step 800, training accuracy 0.96
step 900, training accuracy 0.96
step 1000, training accuracy 0.98
step 1100, training accuracy 0.9
step 1200, training accuracy 0.94
step 1300, training accuracy 0.94
step 1400, training accuracy 0.96
step 1500, training accuracy 0.94
step 1600, training accuracy 0.98
step 1700, training accuracy 1
step 1800, training accuracy 0.98
step 1900, training accuracy 0.98
step 2000, training accuracy 0.96
step 2100, training accuracy 0.98
step 2200, training accuracy 0.98
step 2300, training accuracy 1
step 2400, training accuracy 1
step 2500, training accuracy 1
step 2600, training accuracy 0.98
step 2700, training accuracy 1
step 2800, training accuracy 1
step 2900, training accuracy 0.96
step 3000, tr